##  LLM HuggingFace Serve

In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("llm-ci")

In [ ]:
llm_function = project.new_function("llm_classification",
                                    kind="huggingfaceserve",
                                    model_name="mymodel",
                                    path="huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
llm_run = llm_function.run("serve", profile="template-a100", wait=True)

Wait for 'service' to be in 'running' state which can identified by the presence of 'service' object in the run.

In [ ]:
service = llm_run.refresh().status.service
service

Once 'Service' object available, proceed by making call to deployed service.

In [ ]:
model_name = "mymodel"
json = {
    "inputs": [
        {
            "name": "input-0",
            "shape": [2],
            "datatype": "BYTES",
            "data": ["Hello, my dog is cute", "I am feeling sad"],
        },
    ]
}

In [ ]:
llm_run.invoke(model_name=model_name, json=json).json()

In [ ]:
from pathlib import Path
Path("src").mkdir(exist_ok=True)

## Pipeline

In [ ]:
%%writefile "src/llm_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(path): 
    with pipeline_context() as pc:
         serve = pc.step(name="serve", function="llm_classification", action="serve", inputs={"path":path})

In [ ]:
workflow = project.new_workflow(name="pipeline_llm",
                                kind="kfp",
                                code_src="src/llm_pipeline.py",
                                handler="myhandler")

In [ ]:
wf_build = workflow.run("build", wait=True)

In [ ]:
wf_run = workflow.run("pipeline", parameters={"path": 'huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english'})